In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------ tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts --------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("CESC", "UCS", "UCEC")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
demg_files <- paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets, "_demg_list.txt")
figo_umsmg_files <- paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets, "_figo_umsmg_list.txt")
figo_nsmg_files <- paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets, "_figo_nsmg_list.txt")
survival_files <- paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets, "_survival_umsmg_list.txt")

In [4]:
save_results <- TRUE

In [5]:
demg_lists <- list()
figo_umsmg_lists <- list()
figo_nsmg_lists <- list()
survival_lists <- list()

for (i in 1:3) {
    demg_lists[[unified_dsets[i]]] <- read_lines(demg_files[i])
    figo_umsmg_lists[[unified_dsets[i]]] <- read_lines(figo_umsmg_files[i])
    figo_nsmg_lists[[unified_dsets[i]]] <- read_lines(figo_nsmg_files[i])
    survival_lists[[unified_dsets[i]]] <- read_lines(survival_files[i])
    
#     figo_umsmg_list <- read_lines(figo_umsmg_files[i])
#     figo_nsmg_list <- read_lines(figo_nsmg_files[i])
#     figo_lists[[unified_dsets[i]]] <- figo_umsmg_list %>%
#         union(figo_nsmg_list)
#     survival_lists[[unified_dsets[i]]] <- read_lines(survival_files[i])
}
matrisome_df <- load_matrisome_df(matrisome_path)


-- Column specification -----------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)




In [6]:
l1 <- list(a = c(1, 2, 3), b = c(10, 11, 12))
l2 <- list(a = c(3, 4, 5), b = c(12, 13, 14, 15))

In [7]:
mapply(union, l1, l2)

$a
[1] 1 2 3 4 5

$b
[1] 10 11 12 13 14 15

In [8]:
figo_sig_lists <- mapply(intersect, mapply(union, figo_umsmg_lists, figo_nsmg_lists), demg_lists)
figo_sig_lists %>% map(length)

$unified_cervical_data
[1] 251

$unified_uterine_data
[1] 177

$unified_uterine_endometrial_data
[1] 207

In [9]:
survival_sig_lists <- mapply(intersect, survival_lists, demg_lists)
survival_sig_lists %>% map(length)

$unified_cervical_data
[1] 33

$unified_uterine_data
[1] 18

$unified_uterine_endometrial_data
[1] 26

In [10]:
figo_survival_sig_lists <- mapply(intersect, figo_sig_lists, survival_sig_lists)
figo_survival_sig_lists %>% map(length)

$unified_cervical_data
[1] 13

$unified_uterine_data
[1] 3

$unified_uterine_endometrial_data
[1] 9

In [11]:
demg_all_list <- Reduce(intersect, demg_lists)
demg_all_list %>% length()

[1] 340

In [12]:
figo_all_sig_list <- Reduce(intersect, figo_sig_lists)
figo_all_sig_list %>% length()

[1] 19

In [13]:
survival_all_sig_list <- Reduce(intersect, survival_sig_lists)
survival_all_sig_list %>% length()

[1] 0

In [14]:
demg_all_pairs_list <- demg_lists %>%
    combn(2, simplify = FALSE) %>%
    lapply(function(x) { intersect(x[[1]], x[[2]]) })

demg_all_pairs_list <- setNames(demg_all_pairs_list, c("cesc_ucs", "cesc_ucec", "ucec_ucs"))
demg_all_pairs_list %>% map(length)

$cesc_ucs
[1] 393

$cesc_ucec
[1] 454

$ucec_ucs
[1] 460

In [15]:
figo_all_pairs <- figo_sig_lists %>%
    combn(2, simplify = FALSE) %>%
    lapply(function(x) { intersect(x[[1]], x[[2]]) })

figo_all_pairs <- setNames(figo_all_pairs, c("cesc_ucs", "cesc_ucec", "ucec_ucs"))
figo_all_pairs %>% map(length)

$cesc_ucs
[1] 60

$cesc_ucec
[1] 66

$ucec_ucs
[1] 52

In [16]:
survival_all_pairs <- survival_sig_lists %>%
    combn(2, simplify = FALSE) %>%
    lapply(function(x) { intersect(x[[1]], x[[2]]) })

survival_all_pairs <- setNames(survival_all_pairs, c("cesc_ucs", "cesc_ucec", "ucec_ucs"))
survival_all_pairs %>% map(length)

$cesc_ucs
[1] 0

$cesc_ucec
[1] 1

$ucec_ucs
[1] 0

In [17]:
if (save_results) {
    # FIGO sig
    figo_sig_lists$unified_cervical_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_figo_demg.txt"))
    figo_sig_lists$unified_uterine_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucs_figo_demg.txt"))
    figo_sig_lists$unified_uterine_endometrial_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucec_figo_demg.txt"))
    
    # Survival sig
    survival_sig_lists$unified_cervical_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_survival_demg.txt"))
    survival_sig_lists$unified_uterine_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucs_survival_demg.txt"))
    survival_sig_lists$unified_uterine_endometrial_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucec_survival_demg.txt"))
    
    # FIGO and survival sig
    figo_survival_sig_lists$unified_cervical_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_figo_survival_demg.txt"))
    figo_survival_sig_lists$unified_uterine_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucs_figo_survival_demg.txt"))
    figo_survival_sig_lists$unified_uterine_endometrial_data %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucec_figo_survival_demg.txt"))
    
    # DEMG (all)
    demg_all_list %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/all_demg.txt"))
    
    # FIGO (all)
    figo_all_sig_list %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/all_figo_demg.txt"))
    
    # DEMG pairs
    demg_all_pairs_list$cesc_ucs %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_ucs_demg.txt"))
    demg_all_pairs_list$cesc_ucec %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_ucec_demg.txt"))
    demg_all_pairs_list$ucs_ucec %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucec_ucs_demg.txt"))
    
    # FIGO pairs
    figo_all_pairs$cesc_ucs %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_ucs_figo_demg.txt"))
    figo_all_pairs$cesc_ucec %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_ucec_figo_demg.txt"))
    figo_all_pairs$ucec_ucs %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucec_ucs_figo_demg.txt"))
    
    # Survival pairs
    survival_all_pairs$cesc_ucs %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_ucs_survival_demg.txt"))
    survival_all_pairs$cesc_ucec %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/cesc_ucec_survival_demg.txt"))
    survival_all_pairs$ucec_ucs %>% write_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/ucec_ucs_survival_demg.txt"))
}